In [1]:
import json
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

tqdm.pandas()
pd.options.display.max_columns = None

## Load excel file

In [2]:
kbr_data_path = '../../data/raw/KBR/'
kbr_dir_path = '1_Ankiety_w_gospodarstwach_domowych/'
kbr_file_path = 'Etap_V_1_1_Ankiety_w_gospodarstwach_domowych_Wroclaw.xlsx'
excel_sheet = 'Wrocław_ankiety+podróże'

data = pd.read_excel(os.path.join(kbr_data_path, kbr_dir_path, kbr_file_path), excel_sheet, header=[0,1], engine='openpyxl')

data.columns = [f'{i}_{j}' for i, j in data.columns]
data = data.iloc[0:14241]

if data.shape[1] == len(set(list(data.columns))):
    print('All column names are unique.')
else:
    print('All column names are NOT unique.')

All column names are unique.


In [3]:
data.head()

,Lp_Lp,IDENTYFIKACJA ANKIETY_ID_LOS (numer kwestionariusza),IDENTYFIKACJA ANKIETY_Nr ankietera,IDENTYFIKACJA ANKIETY_Data wypełnienia kwestionariusza,IDENTYFIKACJA ANKIETY_Godzina wypełnienia kwestionariusza,IDENTYFIKACJA ANKIETY_Nr rejonu komunikacyjnego,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Wprowadzenie opłat za wjazd do centrum dla wszystkich pojazdów,"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dopuszczenie dojazdu do centrum dla pojazdów, które nie zanieczyszczają środowiska (np. pojazdy elektryczne, hybrydowe lub spełniające najwyższe normy spalania)","PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jaka jest Pana/Pani opinia na temat rozwiązań wspierających kursowanie tramwajów i autobusów, takich jak priorytet przejazdu na skrzyżowaniach oraz wydzielone pasy na jezdniach?",PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Z jakiego środka transportu zbiorowego korzysta Pan/Pani najczęściej?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dlaczego? \n[tramwaj/autobus],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dlaczego? [Inne]\n[tramwaj/autobus],"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jaka jest Pana/Pani opinia na temat rozwoju systemu kolei aglomeracyjnej na terenie Wrocławia i wokół niego (rozbudowa układu torowego, zwiększenie częstotliwości itd.)?",PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia czystość taboru komunikacji zbiorowej we Wrocławiu?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia punktualność komunikacji zbiorowej we Wrocławiu?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i parkingów rowerowych)?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niekorzystne ustawienia sygnalizacji świetlnej],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [brak chodników i konieczność poruszania się jezdnią/poboczem/wydeptaną ścieżką],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zły stan nawierzchni chodników],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zastawianie chodników przez parkujące samochody],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niebezpieczne zachowania kierowców],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zagrożenie ze strony rowerzystów poruszających się chodnikami],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zbyt wysokie krawężniki],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [brak bieżącego utrzymania czystości/odśnieżania],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niewystarczająca liczba przejść dla pieszych],"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wska

## Dataframe filtering by selected columns + mapping age_bracket and sex values + some processing

In [80]:
chosen_cols = [
    'IDENTYFIKACJA ANKIETY_ID_LOS (numer kwestionariusza)',
    'IDENTYFIKACJA OSOBY_Nr. osoby w gosp. dom.',
    'IDENTYFIKACJA ANKIETY_Data wypełnienia kwestionariusza',
    'DANE O RESPONDENCIE_Przedział wiekowy',
    'DANE O RESPONDENCIE_Płeć',
    'OPIS PODRÓŻY "ŹRÓDŁO"_Godzina rozpoczęcia podróży\n[gg:mm:ss]',
    'OPIS PODRÓŻY "ŹRÓDŁO"_Z jakiego miejsca',
    'OPIS PODRÓŻY "CEL"_Do jakiego miejsca',
    'OPIS PODRÓŻY "CEL"_Do jakiego miejsca [Inne]',
    '23:59:00_motywacje'
]

data_filtered = data[chosen_cols]

data_filtered.columns = [
    'id', 
    'household_person_id',
    'questionnaire_date',
    'age_bracket', 
    'sex',
    'travel_start_time',
    'source', 
    'destination',
    'what_inne',
    'source-destination'
]

data_filtered.fillna(0, inplace=True)
# data_filtered.dropna(inplace=True)
# data_filtered.reset_index(drop=True, inplace=True)

In [81]:
data_filtered.head()

,id,household_person_id,questionnaire_date,age_bracket,sex,travel_start_time,source,destination,what_inne,source-destination
0,LOS_67-67-306,1.0,2018-04-25,61 i więcej (emeryci kobiety),Kobieta,09:00:00,dom,Inne,SKLEP,dom-Inne
1,LOS_67-67-306,1.0,2018-04-25,61 i więcej (emeryci kobiety),Kobieta,09:40:00,Inne,dom,0,Inne-dom
2,LOS_67-67-109,0.0,2018-04-26,20-24 (wiek studencki),Kobieta,0,0,0,0,-
3,LOS_67-67-109,0.0,2018-04-26,20-24 (wiek studencki),Mężczyzna,0,0,0,0,-
4,LOS_67-67-401,0.0,2018-04-26,61 i więcej (emeryci kobiety),Kobieta,0,0,0,0,-


In [82]:
age_bracket_map_dict = {
    '6-15 (dzieci)': '6-15',
    '16-19 (młodzież)': '16-19',
    '20-24 (wiek studencki)': '20-24',
    '25-44 (młodsi pracownicy)': '25-44',
    '45-60 (starsi pracownicy kobiety)': '45-60',  # '45-60(K)',
    '45-65 (starsi pracownicy mężczyźni)': '45-65',  # '45-65(M)',
    '61 i więcej (emeryci kobiety)': '61-x',  # '61-(K)',
    '66 i więcej (emeryci mężczyźni)': '66-x'  # '66-(M)'
}
data_filtered.replace({'age_bracket': age_bracket_map_dict}, inplace=True)

sex_map_dict = {
    'Kobieta': 'K',
    'Mężczyzna': 'M'
}
data_filtered.replace({'sex': sex_map_dict}, inplace=True)

data_filtered['age_sex_comb'] = data_filtered['age_bracket'] + '_' + data_filtered['sex']

data_filtered['person_id'] = data_filtered['id'] + data_filtered['questionnaire_date'].astype('str') + data_filtered['household_person_id'].astype('str')

data_filtered['travel_start_time'] = data_filtered['travel_start_time'].astype('str')

data_filtered['source'] = data_filtered['source'].str.replace('Inne', 'inne')
data_filtered['destination'] = data_filtered['destination'].str.replace('Inne', 'inne')
data_filtered['what_inne'] = data_filtered['what_inne'].str.replace('Inne', 'inne')
data_filtered['source-destination'] = data_filtered['source-destination'].str.replace('Inne', 'inne')


In [83]:
def add_was_travel(row):
    row['was_travel'] = 0 if row['travel_start_time'] == '0' else 1
    return row

data_filtered = data_filtered.progress_apply(add_was_travel, axis=1)

100%|██████████| 14241/14241 [00:10<00:00, 1369.72it/s]


In [84]:
data_filtered.head()

,id,household_person_id,questionnaire_date,age_bracket,sex,travel_start_time,source,destination,what_inne,source-destination,age_sex_comb,person_id,was_travel
0,LOS_67-67-306,1.0,2018-04-25,61-x,K,09:00:00,dom,inne,SKLEP,dom-inne,61-x_K,LOS_67-67-3062018-04-251.0,1
1,LOS_67-67-306,1.0,2018-04-25,61-x,K,09:40:00,inne,dom,NaN,inne-dom,61-x_K,LOS_67-67-3062018-04-251.0,1
2,LOS_67-67-109,0.0,2018-04-26,20-24,K,0,NaN,NaN,NaN,-,20-24_K,LOS_67-67-1092018-04-260.0,0
3,LOS_67-67-109,0.0,2018-04-26,20-24,M,0,NaN,NaN,NaN,-,20-24_M,LOS_67-67-1092018-04-260.0,0
4,LOS_67-67-401,0.0,2018-04-26,61-x,K,0,NaN,NaN,NaN,-,61-x_K,LOS_67-67-4012018-04-260.0,0


## Distribution: number of travels grouped by combinations of age group and gender

### Prepare data

In [85]:
len(data_filtered['person_id'].unique())

6872

In [86]:
data_grouped = data_filtered.groupby(['person_id', 'age_sex_comb', 'was_travel'])['person_id'].count().reset_index(name='travel_amount')

data_grouped.shape[0]

7304

In [87]:
data_grouped.head()

,person_id,age_sex_comb,was_travel,travel_amount
0,LOS_100-100-13052018-06-141.0,61-x_K,1,2
1,LOS_100-100-1642018-06-221.0,25-44_K,1,1
2,LOS_100-100-1642018-06-221.0,45-60_K,0,1
3,LOS_100-100-22812018-06-221.0,25-44_K,1,2
4,LOS_100-100-482018-06-221.0,20-24_K,1,2


In [88]:
# data_grouped[data_grouped['was_travel'] == 0]['travel_amount'].value_counts()

In [89]:
def update_travel_amount(row):
    row['travel_amount'] = 0 if row['was_travel'] == 0 else row['travel_amount']
    return row

data_grouped = data_grouped.progress_apply(update_travel_amount, axis=1)

100%|██████████| 7304/7304 [00:00<00:00, 14101.64it/s]


In [90]:
data_grouped = data_grouped.groupby(['age_sex_comb', 'travel_amount'])['travel_amount'].count().reset_index(name='count')

data_grouped.shape[0]

86

In [91]:
data_grouped.head()

,age_sex_comb,travel_amount,count
0,16-19_K,0,20
1,16-19_K,1,2
2,16-19_K,2,69
3,16-19_K,3,4
4,16-19_K,4,6


### Distribution

In [92]:
unique_age_sex_comb = data_grouped['age_sex_comb'].unique()
max_travel_amount = 6   # data_grouped['travel_amount'].max()

travel_amount_distr = {}

for age_sex_comb in unique_age_sex_comb:
    travel_amount_distr[age_sex_comb] = {}

    for travel_amount in range(0, max_travel_amount+1):
        try:
            count = int(data_grouped[
                (data_grouped['travel_amount'] == travel_amount) & 
                (data_grouped['age_sex_comb'] == age_sex_comb)
            ]['count'])
        except TypeError:
            count = 1

        travel_amount_distr[age_sex_comb][travel_amount] = count

In [93]:
travel_amount_distr

{'16-19_K': {0: 20, 1: 2, 2: 69, 3: 4, 4: 6, 5: 1, 6: 1},
 '16-19_M': {0: 11, 1: 4, 2: 103, 3: 8, 4: 6, 5: 1, 6: 2},
 '20-24_K': {0: 46, 1: 9, 2: 183, 3: 28, 4: 23, 5: 1, 6: 2},
 '20-24_M': {0: 58, 1: 10, 2: 174, 3: 20, 4: 23, 5: 2, 6: 4},
 '25-44_K': {0: 196, 1: 27, 2: 764, 3: 84, 4: 76, 5: 8, 6: 9},
 '25-44_M': {0: 174, 1: 25, 2: 762, 3: 69, 4: 46, 5: 8, 6: 4},
 '45-60_K': {0: 162, 1: 19, 2: 531, 3: 49, 4: 35, 5: 8, 6: 1},
 '45-65_M': {0: 174, 1: 29, 2: 636, 3: 41, 4: 39, 5: 3, 6: 1},
 '6-15_K': {0: 68, 1: 4, 2: 269, 3: 10, 4: 23, 5: 1, 6: 1},
 '6-15_M': {0: 40, 1: 5, 2: 223, 3: 10, 4: 11, 5: 1, 6: 1},
 '61-x_K': {0: 393, 1: 27, 2: 686, 3: 22, 4: 36, 5: 5, 6: 1},
 '66-x_M': {0: 258, 1: 9, 2: 360, 3: 16, 4: 18, 5: 2, 6: 1}}

### Probability distribution

In [94]:
for age_sex_comb, travel_amounts in travel_amount_distr.items():
    travel_amounts_sum = np.array(list(travel_amounts.values())).sum()
    if travel_amounts_sum != 0:
        for i in list(travel_amount_distr[age_sex_comb].keys()):
            travel_amount_distr[age_sex_comb][i] = travel_amount_distr[age_sex_comb][i] / travel_amounts_sum

In [95]:
for age_sex_comb, travel_amounts in travel_amount_distr.items():
    print(age_sex_comb, end='\t -  ')
    print(travel_amounts)
    
# travel_amount_distr

16-19_K	 -  {0: 0.1941747572815534, 1: 0.019417475728155338, 2: 0.6699029126213593, 3: 0.038834951456310676, 4: 0.05825242718446602, 5: 0.009708737864077669, 6: 0.009708737864077669}
16-19_M	 -  {0: 0.08148148148148149, 1: 0.02962962962962963, 2: 0.762962962962963, 3: 0.05925925925925926, 4: 0.044444444444444446, 5: 0.007407407407407408, 6: 0.014814814814814815}
20-24_K	 -  {0: 0.15753424657534246, 1: 0.030821917808219176, 2: 0.6267123287671232, 3: 0.0958904109589041, 4: 0.07876712328767123, 5: 0.003424657534246575, 6: 0.00684931506849315}
20-24_M	 -  {0: 0.19931271477663232, 1: 0.03436426116838488, 2: 0.5979381443298969, 3: 0.06872852233676977, 4: 0.07903780068728522, 5: 0.006872852233676976, 6: 0.013745704467353952}
25-44_K	 -  {0: 0.16838487972508592, 1: 0.023195876288659795, 2: 0.6563573883161512, 3: 0.07216494845360824, 4: 0.06529209621993128, 5: 0.006872852233676976, 6: 0.007731958762886598}
25-44_M	 -  {0: 0.15992647058823528, 1: 0.02297794117647059, 2: 0.7003676470588235, 3: 0.

### Save probability distribution to .json

In [52]:
out_dir = '../../data/processed/mc/travel_planning'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [53]:
out_file = 'travels_num_dist.json'
out_path = os.path.join(out_dir, out_file)

with open(out_path, 'w') as f:
    json.dump(travel_amount_distr, f)

## Distribution: travel purpose grouped by combinations (age group + gender) and type of starting place

### Prepare data

In [96]:
data_grouped = data_filtered.groupby(['age_sex_comb', 'source', 'destination'])['destination'].count().reset_index(name='count')

data_grouped.shape[0]

171

In [97]:
data_grouped.head()

,age_sex_comb,source,destination,count
0,16-19_K,dom,inne,18
1,16-19_K,dom,praca,6
2,16-19_K,dom,szkoła,56
3,16-19_K,dom,uczelnia,12
4,16-19_K,inne,dom,17


In [98]:
# data_filtered[
#     (data_filtered['age_sex_comb'] == '16-19_K') &
#     (data_filtered['source'] == 'dom')
# ]['destination'].value_counts()

In [99]:
data_grouped['source'] = data_grouped['source'].str.replace('szkoła', 'szkola')
data_grouped['destination'] = data_grouped['destination'].str.replace('szkoła', 'szkola')

In [100]:
data_grouped.head()

,age_sex_comb,source,destination,count
0,16-19_K,dom,inne,18
1,16-19_K,dom,praca,6
2,16-19_K,dom,szkola,56
3,16-19_K,dom,uczelnia,12
4,16-19_K,inne,dom,17


### Distribution

In [104]:
unique_age_sex_comb = data_grouped['age_sex_comb'].unique()
unique_place_types = sorted(data_grouped['source'].unique())

travel_purpose_distr = {}

for age_sex_comb in unique_age_sex_comb:
    travel_purpose_distr[age_sex_comb] = {}

    for source_place_type in unique_place_types:
        travel_purpose_distr[age_sex_comb][source_place_type] = {}

        for destination_place_type in unique_place_types:
            try:
                count = int(data_grouped[
                    (data_grouped['age_sex_comb'] == age_sex_comb) &
                    (data_grouped['source'] == source_place_type) &
                    (data_grouped['destination'] == destination_place_type)
                ]['count'])
            except TypeError:
                count = 1

            travel_purpose_distr[age_sex_comb][source_place_type][destination_place_type] = count

In [105]:
for age_sex_comb, source_place_types in travel_purpose_distr.items():
    for source_place_type, destination_place_types in source_place_types.items():
        print(age_sex_comb, end='  -  ')
        print(source_place_type, end='  \t')
        print(destination_place_types)

# travel_purpose_distr

16-19_K  -  dom  	{'dom': 1, 'inne': 18, 'praca': 6, 'szkola': 56, 'uczelnia': 12}
16-19_K  -  inne  	{'dom': 17, 'inne': 6, 'praca': 1, 'szkola': 2, 'uczelnia': 1}
16-19_K  -  praca  	{'dom': 5, 'inne': 1, 'praca': 1, 'szkola': 1, 'uczelnia': 1}
16-19_K  -  szkola  	{'dom': 55, 'inne': 3, 'praca': 1, 'szkola': 1, 'uczelnia': 1}
16-19_K  -  uczelnia  	{'dom': 11, 'inne': 1, 'praca': 1, 'szkola': 1, 'uczelnia': 1}
16-19_M  -  dom  	{'dom': 1, 'inne': 22, 'praca': 5, 'szkola': 92, 'uczelnia': 12}
16-19_M  -  inne  	{'dom': 31, 'inne': 5, 'praca': 1, 'szkola': 1, 'uczelnia': 1}
16-19_M  -  praca  	{'dom': 4, 'inne': 1, 'praca': 1, 'szkola': 1, 'uczelnia': 1}
16-19_M  -  szkola  	{'dom': 83, 'inne': 7, 'praca': 1, 'szkola': 1, 'uczelnia': 1}
16-19_M  -  uczelnia  	{'dom': 11, 'inne': 1, 'praca': 1, 'szkola': 1, 'uczelnia': 1}
20-24_K  -  dom  	{'dom': 2, 'inne': 82, 'praca': 53, 'szkola': 12, 'uczelnia': 135}
20-24_K  -  inne  	{'dom': 110, 'inne': 26, 'praca': 1, 'szkola': 1, 'uczelnia': 

### Probability distribution

In [106]:
for age_sex_comb, source_place_types in travel_purpose_distr.items():
    for source_place_type, destination_place_types in source_place_types.items():
        count_sum = np.array(list(destination_place_types.values())).sum()
        if count_sum != 0:
            for i in list(travel_purpose_distr[age_sex_comb][source_place_type].keys()):
                travel_purpose_distr[age_sex_comb][source_place_type][i] = travel_purpose_distr[age_sex_comb][source_place_type][i] / count_sum

In [107]:
# travel_purpose_distr

for age_sex_comb, source_place_types in travel_purpose_distr.items():
    for source_place_type, destination_place_types in source_place_types.items():
        print(age_sex_comb, end='  -  ')
        print(source_place_type, end='  \t')
        print(destination_place_types)

16-19_K  -  dom  	{'dom': 0.010752688172043012, 'inne': 0.1935483870967742, 'praca': 0.06451612903225806, 'szkola': 0.6021505376344086, 'uczelnia': 0.12903225806451613}
16-19_K  -  inne  	{'dom': 0.6296296296296297, 'inne': 0.2222222222222222, 'praca': 0.037037037037037035, 'szkola': 0.07407407407407407, 'uczelnia': 0.037037037037037035}
16-19_K  -  praca  	{'dom': 0.5555555555555556, 'inne': 0.1111111111111111, 'praca': 0.1111111111111111, 'szkola': 0.1111111111111111, 'uczelnia': 0.1111111111111111}
16-19_K  -  szkola  	{'dom': 0.9016393442622951, 'inne': 0.04918032786885246, 'praca': 0.01639344262295082, 'szkola': 0.01639344262295082, 'uczelnia': 0.01639344262295082}
16-19_K  -  uczelnia  	{'dom': 0.7333333333333333, 'inne': 0.06666666666666667, 'praca': 0.06666666666666667, 'szkola': 0.06666666666666667, 'uczelnia': 0.06666666666666667}
16-19_M  -  dom  	{'dom': 0.007575757575757576, 'inne': 0.16666666666666666, 'praca': 0.03787878787878788, 'szkola': 0.696969696969697, 'uczelnia':

### Save probability distribution to .json

In [26]:
out_dir = '../../data/processed/mc/travel_planning'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [27]:
out_file = 'dest_type_dist.json'
out_path = os.path.join(out_dir, out_file)

with open(out_path, 'w') as f:
    json.dump(travel_purpose_distr, f)

## Distribution: travel start hour grouped by combinations (age group + gender) and type of destinaton place

### Prepare data

In [108]:
data_filtered.shape

(14241, 13)

In [109]:
data_filtered = data_filtered[
    data_filtered['travel_start_time'] != '0'
]

In [110]:
data_filtered.shape

(12598, 13)

In [111]:
data_filtered['travel_start_hour'] = data_filtered['travel_start_time'].astype('str').apply(lambda x: x.split(':')[0])

In [112]:
data_filtered['travel_start_hour'].unique()

array(['09', '18', '19', '16', '12', '13', '06', '14', '05', '07', '08',
       '15', '10', '11', '17', '20', '04', '21', '22', '23', '03', '01',
       '00', '1899-12-30 00'], dtype=object)

In [113]:
data_filtered = data_filtered[data_filtered['travel_start_hour'] != '1899-12-30 00']
data_filtered['travel_start_hour'] = data_filtered['travel_start_hour'].astype('int')

data_filtered['travel_start_hour'].unique()

array([ 9, 18, 19, 16, 12, 13,  6, 14,  5,  7,  8, 15, 10, 11, 17, 20,  4,
       21, 22, 23,  3,  1,  0])

In [119]:
data_grouped = data_filtered.groupby(['destination', 'travel_start_hour'])['travel_start_hour'].count().reset_index(name='count')

data_grouped.shape[0]

87

In [124]:
data_grouped['destination'] = data_grouped['destination'].str.replace('szkoła', 'szkola')

### Distribution

In [132]:
unique_place_types = sorted(data_grouped['destination'].unique())

start_hour_dist = {}

for place_type in unique_place_types:
    start_hour_dist[place_type] = {}

    if place_type in ['uczelnia', 'szkola']:
        min_hour = 6
        max_hour = 19
    else:
        min_hour = 4
        max_hour = 23

    for hour in range(min_hour, max_hour+1):
        try:
            count = int(data_grouped[
                (data_grouped['travel_start_hour'] == hour) & 
                (data_grouped['destination'] == place_type)
            ]['count'])
        except TypeError:
            count = 1

        start_hour_dist[place_type][hour] = count

In [136]:
for place_type, hours in start_hour_dist.items():
    print(place_type, end=' -  ')
    print(hours)
    
# start_hour_dist

dom -  {4: 1, 5: 2, 6: 5, 7: 36, 8: 65, 9: 135, 10: 192, 11: 284, 12: 368, 13: 442, 14: 699, 15: 814, 16: 924, 17: 680, 18: 522, 19: 341, 20: 189, 21: 99, 22: 76, 23: 19}
inne -  {4: 3, 5: 18, 6: 54, 7: 152, 8: 201, 9: 331, 10: 409, 11: 302, 12: 217, 13: 175, 14: 158, 15: 229, 16: 259, 17: 248, 18: 162, 19: 94, 20: 19, 21: 13, 22: 4, 23: 1}
praca -  {4: 25, 5: 182, 6: 567, 7: 864, 8: 468, 9: 162, 10: 46, 11: 37, 12: 27, 13: 52, 14: 16, 15: 9, 16: 10, 17: 9, 18: 7, 19: 2, 20: 2, 21: 11, 22: 1, 23: 1}
szkola -  {6: 15, 7: 609, 8: 82, 9: 30, 10: 7, 11: 7, 12: 7, 13: 10, 14: 13, 15: 11, 16: 5, 17: 2, 18: 1, 19: 1}
uczelnia -  {6: 8, 7: 102, 8: 96, 9: 68, 10: 42, 11: 12, 12: 8, 13: 6, 14: 3, 15: 3, 16: 3, 17: 2, 18: 1, 19: 1}


### Probability distribution

In [137]:
for place_type, hours in start_hour_dist.items():
    hours_sum = np.array(list(hours.values())).sum()
    if hours_sum != 0:
        for i in list(start_hour_dist[place_type].keys()):
            start_hour_dist[place_type][i] = start_hour_dist[place_type][i] / hours_sum

In [139]:
for place_type, hours in start_hour_dist.items():
    print(place_type, end=' -  ')
    print(hours, end='\n\n')
    
# start_hour_dist

dom -  {4: 0.00016969285593076533, 5: 0.00033938571186153065, 6: 0.0008484642796538266, 7: 0.006108942813507551, 8: 0.011030035635499745, 9: 0.02290853555065332, 10: 0.03258102833870694, 11: 0.04819277108433735, 12: 0.06244697098252164, 13: 0.07500424232139827, 14: 0.11861530629560496, 15: 0.13812998472764296, 16: 0.15679619888002716, 17: 0.11539114203292042, 18: 0.08857967079585949, 19: 0.05786526387239097, 20: 0.03207194977091465, 21: 0.016799592737145767, 22: 0.012896657050738164, 23: 0.003224164262684541}

inne -  {4: 0.0009839291571006887, 5: 0.005903574942604133, 6: 0.0177107248278124, 7: 0.0498524106264349, 8: 0.06592325352574614, 9: 0.10856018366677599, 10: 0.1341423417513939, 11: 0.09904886848146933, 12: 0.07117087569694983, 13: 0.05739586749754018, 14: 0.05182026894063627, 15: 0.07510659232535258, 16: 0.08494588389635946, 17: 0.08133814365365694, 18: 0.05313217448343719, 19: 0.03082978025582158, 20: 0.006231551328304362, 21: 0.004263693014102984, 22: 0.0013119055428009183, 23

### Save probability distribution to .json

In [140]:
out_dir = '../../data/processed/mc/travel_planning'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [141]:
out_file = 'start_hour_dist.json'
out_path = os.path.join(out_dir, out_file)

with open(out_path, 'w') as f:
    json.dump(start_hour_dist, f)